In [99]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle
import string
import re
import nltk
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
from gensim.parsing.preprocessing import strip_non_alphanum
from gensim.parsing.preprocessing import strip_numeric
from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split



 # Data Cleaning 

In [177]:

# training data containing labels 
data = pd.read_csv('/Users/William/Desktop/CS247/Project/groupproject/jigsaw-toxic-comment-classification-challenge/train.csv')


# test data to use when testing the model only 
test_labels = pd.read_csv('/Users/William/Desktop/CS247/Project/groupproject/jigsaw-toxic-comment-classification-challenge/test_labels.csv')
test_data =   pd.read_csv('/Users/William/Desktop/CS247/Project/groupproject/jigsaw-toxic-comment-classification-challenge/test.csv')


 #  join labels and test_data 
test_data = test_data.join(test_labels,lsuffix='left', rsuffix='right')

test_data = test_data.loc[(test_data['toxic'] != -1)  & (test_data['severe_toxic'] != -1) & (test_data['obscene'] != -1) & (test_data['threat'] != -1) & (test_data['insult'] != -1) & (test_data['identity_hate'] != -1),['idleft','comment_text','toxic','severe_toxic','obscene','threat','insult','identity_hate']]


In [178]:
# data['Target'] = data.toxic.astype(str) + data.severe_toxic.astype(str) + data.obscene.astype(str) + data.threat.astype(str) +  data.insult.astype(str) + data.identity_hate.astype(str)

# test_data['Target'] = test_data.toxic.astype(str) + test_data.severe_toxic.astype(str) + test_data.obscene.astype(str) + test_data.threat.astype(str) +  test_data.insult.astype(str) + test_data.identity_hate.astype(str)

data['Target'] = ''
for i in range(len(data)):
    data['Target'][i] = list([data.toxic[i],data.severe_toxic[i],data.obscene[i],data.threat[i],data.insult[i],data.identity_hate[i]])
    

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [189]:
test_data['Target'] = ''
for i in range(len(test_data)):
    test_data['Target'].iloc[i] = list([test_data.toxic.iloc[i],test_data.severe_toxic.iloc[i],test_data.obscene.iloc[i],test_data.threat.iloc[i],test_data.insult.iloc[i],test_data.identity_hate.iloc[i] ] )

/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


### Helper functions 

In [191]:
#gobal
PunctChars = r'''[`'“".?!,:;]'''
Punct = '%s+' % PunctChars
Entity = '&(amp|lt|gt|quot);'
printable = set(string.printable)


#  dictionary of possible  word expansions 

apostrophe_expansions = {
"y": "why",
"u": "you",
"r": "are",
"luv": "love",
"&" : "and",
"idk" : "i do not know",
"imo" : "in my opinion",
"ain't": "are not",
"aren't": "are not ",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he had ",
"he'd've": "he would have",
"he'll": "he will",
"he'll've": "he shall have",
"he's": "he is",
"how'd": "how did",
"how'd'y": "how do you",
"how'll": "how will",
"how's": "how is",
"i'd": "i would",
"i'd've": "i would have",
"i'll": "i will",
"i'll've": "i will have",
"i'm": "i am",
"i've": "i have",
"isn't": "is not",
"it'd": "it would",
"it'd've": "it would have",
"it'll": "it will",
"it'll've": "it will have",
"it's": "it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"mightn't've": "might not have",
"must've": "must have",
"mustn't": "must not",
"mustn't've": "must not have",
"needn't": "need not",
"needn't've": "need not have",
"o'clock": "of the clock",
"oughtn't": "ought not",
"oughtn't've": "ought not have",
"shan't": "shall not",
"sha'n't": "shall not",
"shan't've": "shall not have",
"she'd": "she would",
"she'd've": "she would have",
"she'll": "she will",
"she'll've": "she will have",
"she's": "she is",
"should've": "should have",
"shouldn't": "should not",
"shouldn't've": "should not have",
"that'd": "that would",
"that'd've": "that would have",
"that's": "that is",
"there'd": "there would",
"there'd've": "there would have",
"there's": "there is",
"they'd": "they would",
"they'd've": "they would have",
"they'll": "they will",
"they'll've": " they will have",
"they're": "they are",
"they've": "they have",
"to've": "to have",
"wasn't": "was not",
"we'd": "we would",
"we'd've": "we would have",
"we'll": "we will",
"we'll've": "we will have",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what shall",
"what'll've": "what shall have",
"what're": "what are",
"what's": "what is",
"what've": "what have",
"when's": " when is",
"when've": "when have",
"where'd": "where did",
"where's": "where is",
"where've": "where have",
"who'll": "who will",
"who'll've": "who will have",
"who's": "who is",
"who've": "who have",
"why's": "why is",
"why've": "why have",
"will've": "will have",
"won't": "will not",
"won't've": "will not have",
"would've": "would have",
"wouldn't": "would not",
"wouldn't've": "would not have",
"y'all": "you all",
"y'all'd": "you all would",
"y'all'd've": "you all would have",
"y'all're": "you all are",
"y'all've": "you all have",
"you'd": "you would",
"you'd've": "you would have",
"you'll": "you will",
"you'll've": "you will have",
"you're": "you are",
"you've": "you have"
}

def contractions(text):    
           return  " ".join([ apostrophe_expansions[word] if word in apostrophe_expansions else word for word in text.split() ] )


In [192]:
# # helper functions
# def regex_or(*items):
#     r = '|'.join(items)
#     r = '(' + r + ')'
#     return r

# def pos_lookahead(r):
#     return '(?=' + r + ')'

# def neg_lookahead(r):
#     return '(?!' + r + ')'

# def optional(r):
#     return '(%s)?' % r

def trim(text):
    ''' 
    trim leading and trailing spaces in the  text
    '''
    return text.strip()

def strip_whiteSpaces(text):
    '''
    Strip all white spaces
    '''
    text = re.sub(r'[\s]+', ' ', text)
    return text

def strip_unicode(text):
    '''
    Strip all unicode characters from text
    '''
    text = ''.join(i for i in text if ord(i)<128)
    return text 


def remove_nonalpha_char(text):
    return  strip_non_alphanum(text)

def remove_numeric_char(text):
     return strip_numeric(text)
    
def to_LowerCase(text):
    '''
    Convert  text to lower case alphabets
    '''
    text = text.lower()
    return text


def prune_multple_consecutive_same_char(text):
    '''
    yesssssssss  is converted to yes
    '''
    text = re.sub(r'(.)\1+', r'\1\1', text)
    return text



def process_words(text):
    #remove words in brackets 
    text = re.sub('\[.*?\]', '', text)
    #remove words containing numbers 
    text = re.sub('\w*\d\w*', '', text)
    #remove punctuations
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    return text


def clean_text(text):
    '''Get rid of some additional punctuation and non-sensical text that was missed the first time around.'''
    text = re.sub('[‘’“”…]', '', text)
    text = re.sub('\n', '', text)
    return text


def remove_stop_words(text):
     return " ".join([i for i in text.split() if not i in stop_words])


### Basic Preprocessing of text data

In [193]:
def basic_preprocessing(text):
    text = to_LowerCase(text)
    text = contractions(text)
    text = strip_unicode(text)
    text = trim(text)
    text = strip_whiteSpaces(text)   
    text = prune_multple_consecutive_same_char(text)
    text = remove_nonalpha_char(text)
    text = remove_numeric_char(text)
    text = process_words(text)
    text = clean_text(text)
    text = trim(text)
    text = strip_whiteSpaces(text)
    return text

# preprocessing for words embeddings 
def pre_word_emb(text):
    text = basic_preprocessing(text)
    text = remove_stop_words(text)
    return text

# preprocessing for sentence embeddings
def pre_sentence_emb(text):
    text = basic_preprocessing(text)
    return text

### Tokenisation

In [194]:
#  lamda functions to apply to our training data 
word_preprocessing = lambda x: pre_word_emb(x)
sentence_preprocessing = lambda x : pre_sentence_emb(x)
tokenize = lambda x : word_tokenize(x)



### Applying preprocessing 

In [195]:
#  comment text cleaned and tokenize
data['comment_text_wp'] = data.comment_text.apply(word_preprocessing)
data['comment_tokenize'] = data.comment_text_wp.apply(tokenize)
test_data['comment_text_wp'] = test_data.comment_text.apply(word_preprocessing)
test_data['comment_tokenize'] = test_data.comment_text_wp.apply(tokenize)

In [196]:
# comment text cleaned and left as a sentence for sentence embeddings
data['comment'] = data.comment_text.apply(sentence_preprocessing)
test_data['comment'] = test_data.comment_text.apply(sentence_preprocessing)

In [197]:
# split and save data for future use 
train_data , validation_data = train_test_split(data, test_size=0.35, random_state=42)

In [210]:
train_data.to_pickle('train_data.pkl')
validation_data.to_pickle('valid_data.pkl')
test_data.to_pickle('test_data.pkl')
data.to_pickle('data.pkl')